In [1]:
import spark_catalog
import duckdb

In [2]:
catalog = spark_catalog.SparkCatalog(app_name="kexp", lake_location="../../../data/spark/kexp")

:: loading settings :: url = jar:file:/Users/timburns/Documents/PythonProjects/.pyenv/kexp/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/timburns/.ivy2/cache
The jars for the packages stored in: /Users/timburns/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bfd4bd23-d587-4817-88db-a8cec35a6c1f;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 134ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

23/04/12 12:34:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/12 12:34:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/Users/timburns/Documents/PythonProjects/.pyenv/kexp/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

23/04/12 12:34:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
kexp_playlist.show()

In [11]:
catalog.sql("select count(*) as play_count from KEXP_PLAYLIST").show()

In [12]:
catalog.sql("select id, airdate, artist, song, show from KEXP_PLAYLIST order by id desc").show()

In [21]:
catalog.get_data_frame("bronze", "KEXP_PLAYLIST").printSchema()

+------------+-----------------+-----------------------+----------------------+
|program_name|            genre|count(DISTINCT airdate)|count(DISTINCT artist)|
+------------+-----------------+-----------------------+----------------------+
|     90.TEEN|         Eclectic|                     20|                    14|
|     90.TEEN|             Rock|                     20|                    14|
|     90.TEEN|      Variety Mix|                     20|                    14|
|   Audioasis|         Eclectic|                     49|                    38|
|   Audioasis|Pacific Northwest|                     49|                    38|
|   Audioasis|             Rock|                     49|                    38|
|  Drive Time|         Eclectic|                    163|                   119|
|  Drive Time|             Rock|                    163|                   119|
|  Drive Time|      Variety Mix|                    163|                   119|
|       Early|         Eclectic|        

In [22]:
catalog.sql("select id, size(artist_ids) from KEXP_PLAYLIST order by size(artist_ids) desc").show()

+-----------------+-----------------------+----------------------+
|            genre|count(DISTINCT airdate)|count(DISTINCT artist)|
+-----------------+-----------------------+----------------------+
|         Afrobeat|                     33|                    27|
|            Blues|                    115|                    55|
|          Country|                    115|                    55|
|               DJ|                     16|                    12|
|              EBM|                     17|                    16|
|         Eclectic|                   1662|                   952|
|       Electronic|                     82|                    65|
|          Hip Hop|                     48|                    48|
|             Jazz|                     29|                    17|
|             KEXP|                     50|                     6|
|            Latin|                     42|                    33|
|             Live|                      7|                   

In [24]:
catalog.sql("select artist, song, album, artist_ids from KEXP_PLAYLIST where ID = 3177634").show(truncate=False, vertical=True)

In [25]:
kexp_playlist = catalog.get_data_frame("bronze", "KEXP_PLAYLIST")

+--------------------+--------------------+-------------+------------------------+-------------------------+
|                  id|              artist|silver_source|silver_created_timestamp|silver_modified_timestamp|
+--------------------+--------------------+-------------+------------------------+-------------------------+
|25302723-b24f-4d0...|     $wingin’ Utter$|KEXP_PLAYLIST|     2023-04-11 18:37:43|      2023-04-11 18:37:43|
|5e29d25a-ba6f-402...|                 310|KEXP_PLAYLIST|     2023-04-11 18:37:43|      2023-04-11 18:37:43|
|3e63ef4f-93c5-48a...|            38 Spesh|KEXP_PLAYLIST|     2023-04-11 18:37:43|      2023-04-11 18:37:43|
|07832b42-8826-4ab...|               6LACK|KEXP_PLAYLIST|     2023-04-11 18:37:43|      2023-04-11 18:37:43|
|9d7c95f4-9575-4b7...|           79rs Gang|KEXP_PLAYLIST|     2023-04-11 18:37:43|      2023-04-11 18:37:43|
|030c731f-6b7c-407...|            8fatfat8|KEXP_PLAYLIST|     2023-04-11 18:37:43|      2023-04-11 18:37:43|
|48513f24-37d5-423.

In [31]:
from pyspark.sql.functions import explode

In [32]:
kexp_playlist.withColumn("artist_ids", explode(kexp_playlist.artist_ids)).show()

+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+------------+----------------+----------+---------------+-----+--------------------+--------------------+--------------------+--------+--------------------+
|            airdate|               album|              artist|          artist_ids|             comment|     id|           image_uri|is_live|is_local|is_request|           label_ids|              labels|play_type|recording_id|release_date|release_group_id|release_id|rotation_status| show|            show_uri|                song|       thumbnail_uri|track_id|                 uri|
+-------------------+--------------------+--------------------+--------------------+--------------------+-------+--------------------+-------+--------+----------+--------------------+--------------------+---------+------------+-----

In [36]:
artist_df = catalog.sql("select artist, artist_ids from KEXP_PLAYLIST order by artist")

In [37]:
artist_df.withColumn("artist_ids", explode(artist_df.artist_ids)).show()

+--------------------+--------------------+
|              artist|          artist_ids|
+--------------------+--------------------+
|     $wingin’ Utter$|25302723-b24f-4d0...|
|                 310|5e29d25a-ba6f-402...|
|            38 Spesh|3e63ef4f-93c5-48a...|
|               6LACK|07832b42-8826-4ab...|
|           79rs Gang|9d7c95f4-9575-4b7...|
|            8fatfat8|030c731f-6b7c-407...|
|            8fatfat8|030c731f-6b7c-407...|
|9th Wonder feat. ...|48513f24-37d5-423...|
|9th Wonder feat. ...|6b1acf37-0b3b-4cc...|
|9th Wonder feat. ...|6705519c-4f9b-4e5...|
|9umba, Toss & Mdo...|d3253aba-df21-4dc...|
|9umba, Toss & Mdo...|13bcba23-2fd2-4b7...|
|9umba, Toss & Mdo...|936c3773-6467-486...|
|9umba, Toss & Mdo...|7a7b5cfe-24e0-42f...|
|9umba, Toss & Mdo...|40b8dcd1-6efd-49f...|
|9umba, Toss & Mdo...|8d3dd2fe-0aac-4a3...|
|9umba, Toss & Mdo...|e701b534-eea5-4b6...|
|9umba, Toss & Mdo...|7c745985-26bf-444...|
|             A House|c2f465fc-1f81-4d2...|
|A Tribe Called Quest|9689aa5a-4